# <font color=darkblue> Machine Learning model deployment with Flask framework</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with the help of the flask framework.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import sklearn
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, accuracy_score
from sklearn.ensemble import RandomForestRegressor
import pickle
import warnings
warnings.filterwarnings('ignore')

### 2. Load the dataset

In [5]:
df= pd.read_csv(r'D:\elsie\Data Science\lab 5\car+data.csv')
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 3. Check the shape and basic information of the dataset.

In [6]:
df.shape

(301, 9)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [8]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
296    False
297    False
298    False
299    False
300    False
Length: 301, dtype: bool

In [9]:
df[df.duplicated()]

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
17,ertiga,2016,7.75,10.79,43000,Diesel,Dealer,Manual,0
93,fortuner,2015,23.00,30.61,40000,Diesel,Dealer,Automatic,0


In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df[df.duplicated()]

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner


In [17]:
df.shape

(299, 8)

### 5. Drop the columns which you think redundant for the analysis.

In [13]:
df.shape

(299, 9)

In [14]:
df.drop('Car_Name', axis=1, inplace=True)

In [15]:
df.shape

(299, 8)

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [18]:
df['age_of_car'] = 2024 - df['Year']

In [19]:
df.drop('Year', axis = 1, inplace = True)

In [20]:
df.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_car
0,3.35,5.59,27000,Petrol,Dealer,Manual,0,10
1,4.75,9.54,43000,Diesel,Dealer,Manual,0,11
2,7.25,9.85,6900,Petrol,Dealer,Manual,0,7
3,2.85,4.15,5200,Petrol,Dealer,Manual,0,13
4,4.60,6.87,42450,Diesel,Dealer,Manual,0,10


### 7. Encode the categorical columns

In [21]:
df['Fuel_Type'].unique()

array(['Petrol', 'Diesel', 'CNG'], dtype=object)

In [22]:
df['Seller_Type'].unique()

array(['Dealer', 'Individual'], dtype=object)

In [23]:
df['Transmission'].unique()

array(['Manual', 'Automatic'], dtype=object)

In [24]:
df['Fuel_Type'] = df['Fuel_Type'].replace({'Petrol':0, 'Diesel':1, 'CNG':2})
df['Seller_Type'] = df['Seller_Type'].replace({'Dealer':1, 'Individual':2})
df['Transmission'] = df['Transmission'].replace({'Manual':1, 'Automatic':2})

In [25]:
df['Fuel_Type'].unique()

array([0, 1, 2], dtype=int64)

In [26]:
df['Seller_Type'].unique()

array([1, 2], dtype=int64)

In [27]:
df['Transmission'].unique()

array([1, 2], dtype=int64)

In [28]:
df.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_car
0,3.35,5.59,27000,0,1,1,0,10
1,4.75,9.54,43000,1,1,1,0,11
2,7.25,9.85,6900,0,1,1,0,7
3,2.85,4.15,5200,0,1,1,0,13
4,4.60,6.87,42450,1,1,1,0,10


### 8. Separate the target and independent features.

In [29]:
x = df.drop('Selling_Price', axis = 1)
y = df['Selling_Price']

### 9. Split the data into train and test.

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(239, 7) (60, 7)
(239,) (60,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [31]:
rf = RandomForestRegressor()
rf.fit(x_train,y_train)

RandomForestRegressor()

In [32]:
y_train_pred = rf.predict(x_train)
y_test_pred = rf.predict(x_test)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
print(r2_train)
print(r2_test)

0.9898632820212535
0.9576743446574525


### 11. Create a pickle file with an extension as .pkl

In [33]:
import pickle

In [34]:
pickle.dump(rf, open('model.pkl','wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

### b) Create app.py file and write the predict function

In [35]:
from flask import Flask, render_template, request, jsonify
import pickle
import numpy as np
import sklearn
app = Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))
@app.route('/',methods = ['GET'])
def Home():
    return render_template('index.html')
@app.route("/predict", methods = ['POST'])
def predict():
    if request.method =='POST':
        Present_Price = float(request.form['Present_price'])
        kms_Driven = int(request.form['Kms_Driven'])
        Owner = int(request.form['Owner'])
        Fuel_Type = request.form['Fuel_Type']
        Age_of_Car = request.form['Age_of_Car']
        Seller_Type = request.form['Seller_Type']
        Transmission = request.form['Transmission']
        prediction = modl.predict([[Present_price,Kms_Driven, Owner, Age_of_car, Fuel_Type, Seller_Type, Transmission]])
        output = round(predict[0],2)
        return render_template('index.html', prediction_text = 'You can sell your car at {} lakhs'.format(output))
if __name__ =='__main__':
    app.run(debug = True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

### 13. Run the app.py python file which will render to index html page then enter the input values and get the prediction.

### Happy Learning :)